In [19]:
import datetime,time
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc={'figure.figsize':(12,3)})
pd.set_option('display.max_columns', 50)
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split, KFold,cross_val_score,cross_validate,GridSearchCV
from lightgbm import LGBMRegressor,LGBMClassifier
from catboost import CatBoostRegressor,CatBoostClassifier
from xgboost import XGBRegressor,XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder,PolynomialFeatures
le=LabelEncoder()
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import SplineTransformer
plt.style.use("seaborn-whitegrid")
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm_notebook
import os
train = pd.read_csv('data/Train.csv')
test = pd.read_csv('data/Test.csv')

train['ind'] = 1
test['ind'] = 0
df= pd.concat([train, test])
obj=df.select_dtypes('object')
num=df.select_dtypes(exclude='object')
col=df.drop(['ID','power-generated',],axis=1).columns
df#ctb

,ID,distance-to-solar-noon,temperature,wind-direction,wind-speed,sky-cover,visibility,humidity,average-wind-speed-(period),average-pressure-(period),power-generated,ind
0,ID_000C6D,0.168350,52,6,1.1,0,6.0,80,8.0,766.572,22482.0,1
1,ID_001BF0,0.349153,63,26,16.3,4,10.0,81,16.0,760.984,1755.0,1
2,ID_007316,0.857875,63,27,13.9,4,10.0,93,11.0,758.444,0.0,1
3,ID_009D0E,0.150113,70,30,9.7,0,9.0,71,3.0,759.968,27771.0,1
4,ID_00B4A1,0.465986,63,27,16.0,1,10.0,72,16.0,759.460,1748.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
872,ID_FF6826,0.944046,65,28,11.1,1,10.0,81,15.0,760.984,NaN,0
873,ID_FFBF07,0.176672,48,4,3.8,4,9.0,86,6.0,766.318,NaN,0
874,ID_FFC86B,0.560726,61,27,15.1,4,10.0,78,8.0,761.492,NaN,0
875,ID_FFCD82,0.637306,56,18,9.9,3,10.0,64,10.0,758.952,NaN,0


In [20]:
df['power-generated'].loc[df['power-generated'] >=32018]  = 32018
df['interaction'] = df['average-pressure-(period)'] * df['average-wind-speed-(period)']
df['sin_speed'] = np.sin(df['average-wind-speed-(period)'])
df['pressure_wind_diff'] = np.abs(df['average-pressure-(period)'] - df['average-wind-speed-(period)'])
df['time-of-day'] = abs(df['distance-to-solar-noon'] - 0.5)

df['sin_pressure'] = np.sin(df['average-pressure-(period)'])
df['sin_cos_interaction'] = np.sin(df['average-pressure-(period)']) * np.cos(df['average-wind-speed-(period)'])
df['pressure_wind_diff'] = np.abs(df['average-pressure-(period)'] - df['average-wind-speed-(period)'])
df['pressure_wind_distance'] = np.sqrt((df['average-pressure-(period)'] - df['average-wind-speed-(period)']) ** 2)
df['pressure_squared'] = df['average-pressure-(period)'] ** 2
df['wind_speed_squared'] = df['average-wind-speed-(period)'] ** 2

In [21]:
df

,ID,distance-to-solar-noon,temperature,wind-direction,wind-speed,sky-cover,visibility,humidity,average-wind-speed-(period),average-pressure-(period),power-generated,ind,interaction,sin_speed,pressure_wind_diff,time-of-day,sin_pressure,sin_cos_interaction,pressure_wind_distance,pressure_squared,wind_speed_squared
0,ID_000C6D,0.168350,52,6,1.1,0,6.0,80,8.0,766.572,22482.0,1,6132.576,0.989358,758.572,0.331650,0.023390,-0.003403,758.572,587632.631184,64.0
1,ID_001BF0,0.349153,63,26,16.3,4,10.0,81,16.0,760.984,1755.0,1,12175.744,-0.287903,744.984,0.150847,0.658315,-0.630441,744.984,579096.648256,256.0
2,ID_007316,0.857875,63,27,13.9,4,10.0,93,11.0,758.444,0.0,1,8342.884,-0.999990,747.444,0.357875,-0.968757,-0.004287,747.444,575237.301136,121.0
3,ID_009D0E,0.150113,70,30,9.7,0,9.0,71,3.0,759.968,27771.0,1,2279.904,0.141120,756.968,0.349887,-0.293057,0.290124,756.968,577551.361024,9.0
4,ID_00B4A1,0.465986,63,27,16.0,1,10.0,72,16.0,759.460,1748.0,1,12151.360,-0.287903,743.460,0.034014,-0.721123,0.690590,743.460,576779.491600,256.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,ID_FF6826,0.944046,65,28,11.1,1,10.0,81,15.0,760.984,NaN,0,11414.760,0.650288,745.984,0.444046,0.658315,-0.500114,745.984,579096.648256,225.0
873,ID_FFBF07,0.176672,48,4,3.8,4,9.0,86,6.0,766.318,NaN,0,4597.908,-0.279415,760.318,0.323328,-0.228569,-0.219465,760.318,587243.277124,36.0
874,ID_FFC86B,0.560726,61,27,15.1,4,10.0,78,8.0,761.492,NaN,0,6091.936,0.989358,753.492,0.060726,0.941339,-0.136965,753.492,579870.066064,64.0
875,ID_FFCD82,0.637306,56,18,9.9,3,10.0,64,10.0,758.952,NaN,0,7589.520,-0.544021,748.952,0.137306,-0.967062,0.811434,748.952,576008.138304,100.0


In [65]:
#px.box(df['power-generated'])
#df['distance-to-solar-noon'].value_counts()

In [22]:
def summary(df):
    sum = pd.DataFrame(df.dtypes, columns=['dtypes'])
    sum['missing_num'] = df.isna().sum().values*100
    sum['missing%'] = (df.isna().sum().values*100)/len(df)
    sum['uniques'] = df.nunique().values
    sum['count'] = df.count().values
    desc = pd.DataFrame(df.describe().T)
    sum['min'] = desc['min']
    sum['max'] = desc['max']
    sum['mean'] = desc['mean']
    sum['75%'] = desc['75%']
    return sum

summary(df).style.background_gradient(cmap='Blues')

,dtypes,missing_num,missing%,uniques,count,min,max,mean,75%
ID,object,0,0.000000,2920,2920,nan,nan,nan,nan
distance-to-solar-noon,float64,0,0.000000,2660,2920,0.050401,1.141361,0.503294,0.739528
temperature,int64,0,0.000000,37,2920,42.000000,78.000000,58.468493,63.000000
wind-direction,int64,0,0.000000,35,2920,1.000000,36.000000,24.953425,29.000000
wind-speed,float64,0,0.000000,159,2920,1.100000,26.600000,10.096986,13.100000
sky-cover,int64,0,0.000000,5,2920,0.000000,4.000000,1.987671,3.000000
visibility,float64,0,0.000000,17,2920,0.000000,10.000000,9.557705,10.000000
humidity,int64,0,0.000000,75,2920,14.000000,100.000000,73.513699,84.000000
average-wind-speed-(period),float64,100,0.034247,32,2919,0.000000,40.000000,10.129154,15.000000
average-pressure-(period),float64,0,0.000000,90,2920,748.792000,775.462000,762.451111,764.794000


In [23]:
df.fillna(df.mean(),inplace=True)
top20_corrs = abs(df.corr()['power-generated']).sort_values(ascending = False).head(30)
top20_corrs

power-generated                1.000000e+00
distance-to-solar-noon         6.327627e-01
humidity                       4.441620e-01
time-of-day                    2.601840e-01
wind_speed_squared             2.330989e-01
interaction                    2.245512e-01
average-wind-speed-(period)    2.244158e-01
pressure_wind_diff             1.928978e-01
pressure_wind_distance         1.928978e-01
sky-cover                      1.616399e-01
wind-direction                 1.285269e-01
wind-speed                     1.122782e-01
temperature                    1.105439e-01
visibility                     7.216032e-02
sin_speed                      3.439173e-02
pressure_squared               2.996394e-02
average-pressure-(period)      2.984839e-02
sin_cos_interaction            2.136671e-02
sin_pressure                   1.578932e-02
ind                            1.743535e-16
Name: power-generated, dtype: float64

In [24]:
# Trying to remove redundant features
def redundant_feature(df):
    redundant = []
    for i in df.columns:
        counts = df[i].value_counts()
        count_max = counts.iloc[0]
        if count_max / len(df) * 100 > 99:
            redundant.append(i)
    redundant = list(redundant)
    return redundant
redundant_features = redundant_feature(df)
redundant_features

[]

In [25]:
df.replace([np.inf, -np.inf], 0, inplace=True)

In [26]:
train = df[df.ind == 1]
test = df[df.ind == 0]
X=train[col]#.values.reshape(-1,1)
y = train['power-generated']

In [27]:
cnn={'learning_rate': 0.08457828408816369, 'n_estimators': 412, 'max_depth': 8, 'l2_leaf_reg': 0.0018938792811299563, 'subsample': 0.852981565254582, 'colsample_bylevel': 0.6280580147587667, 'min_child_samples': 5}

In [28]:
ctb = CatBoostRegressor(**cnn).fit(X,y)
xgr = XGBRegressor().fit(X,y)
lgr = LGBMRegressor().fit(X,y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

0:	learn: 9480.6215562	total: 6.03ms	remaining: 2.48s
1:	learn: 8832.4455857	total: 13.9ms	remaining: 2.85s
2:	learn: 8216.9652996	total: 20.8ms	remaining: 2.83s
3:	learn: 7657.5844891	total: 27.9ms	remaining: 2.85s
4:	learn: 7142.1802161	total: 35ms	remaining: 2.85s
5:	learn: 6674.8335660	total: 41.6ms	remaining: 2.81s
6:	learn: 6282.5728711	total: 48.9ms	remaining: 2.83s
7:	learn: 5924.6657984	total: 56.4ms	remaining: 2.85s
8:	learn: 5598.4334141	total: 63.8ms	remaining: 2.86s
9:	learn: 5289.8968969	total: 70.3ms	remaining: 2.83s
10:	learn: 5020.1379898	total: 78.4ms	remaining: 2.86s
11:	learn: 4750.1601297	total: 86ms	remaining: 2.87s
12:	learn: 4530.6473419	total: 94.3ms	remaining: 2.89s
13:	learn: 4320.9331983	total: 103ms	remaining: 2.93s
14:	learn: 4142.1782836	total: 111ms	remaining: 2.94s
15:	learn: 3980.6388678	total: 118ms	remaining: 2.91s
16:	learn: 3833.5585628	total: 125ms	remaining: 2.9s
17:	learn: 3697.8381327	total: 132ms	remaining: 2.88s
18:	learn: 3579.0152080	total:

171:	learn: 1113.8087338	total: 1.4s	remaining: 1.96s
172:	learn: 1110.1376193	total: 1.41s	remaining: 1.95s
173:	learn: 1106.3472818	total: 1.42s	remaining: 1.94s
174:	learn: 1102.0381051	total: 1.43s	remaining: 1.93s
175:	learn: 1099.1000278	total: 1.44s	remaining: 1.93s
176:	learn: 1090.1092993	total: 1.45s	remaining: 1.92s
177:	learn: 1087.6829651	total: 1.46s	remaining: 1.91s
178:	learn: 1084.1907837	total: 1.46s	remaining: 1.91s
179:	learn: 1075.6950697	total: 1.47s	remaining: 1.9s
180:	learn: 1068.5749869	total: 1.48s	remaining: 1.89s
181:	learn: 1064.8126551	total: 1.49s	remaining: 1.88s
182:	learn: 1057.4722414	total: 1.49s	remaining: 1.87s
183:	learn: 1048.8519873	total: 1.5s	remaining: 1.86s
184:	learn: 1042.8856560	total: 1.51s	remaining: 1.85s
185:	learn: 1037.7842295	total: 1.52s	remaining: 1.84s
186:	learn: 1032.3735394	total: 1.53s	remaining: 1.84s
187:	learn: 1025.8431393	total: 1.54s	remaining: 1.84s
188:	learn: 1021.5400615	total: 1.55s	remaining: 1.83s
189:	learn: 1

329:	learn: 562.1281585	total: 3.03s	remaining: 754ms
330:	learn: 560.8462587	total: 3.04s	remaining: 744ms
331:	learn: 559.0998380	total: 3.05s	remaining: 735ms
332:	learn: 557.6893995	total: 3.06s	remaining: 725ms
333:	learn: 555.8769071	total: 3.07s	remaining: 716ms
334:	learn: 554.0414163	total: 3.08s	remaining: 707ms
335:	learn: 550.7268256	total: 3.08s	remaining: 698ms
336:	learn: 549.9208737	total: 3.09s	remaining: 688ms
337:	learn: 548.5597711	total: 3.1s	remaining: 679ms
338:	learn: 543.4341536	total: 3.11s	remaining: 669ms
339:	learn: 541.0886390	total: 3.11s	remaining: 659ms
340:	learn: 538.0038142	total: 3.12s	remaining: 650ms
341:	learn: 535.6951418	total: 3.13s	remaining: 640ms
342:	learn: 534.5138529	total: 3.13s	remaining: 631ms
343:	learn: 532.3797080	total: 3.14s	remaining: 621ms
344:	learn: 531.0169950	total: 3.15s	remaining: 612ms
345:	learn: 528.0008818	total: 3.16s	remaining: 602ms
346:	learn: 525.9193094	total: 3.17s	remaining: 593ms
347:	learn: 524.0068231	total

In [29]:
y_predicted = ctb.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_test, (y_predicted)))
print("Root Mean Square Error:\n")
print(RMSE)#1097.92

Root Mean Square Error:

364.62037173035395


In [30]:
# Analyse predictions
pred_errors = X_test.copy()
pred_errors['target'] = y_test
pred_errors['prediction'] = y_predicted
pred_errors['error'] = abs(pred_errors.prediction - pred_errors.target)
pred_errors = pred_errors[[ 'target', 'prediction', 'error']]
pred_errors.sort_values(by = 'error', ascending = False, inplace = True)
pred_errors

,target,prediction,error
1698,11209.0,13451.064235,2242.064235
1693,30094.0,28124.843259,1969.156741
1342,25632.0,27349.028345,1717.028345
203,1661.0,3166.388878,1505.388878
564,1728.0,3206.004575,1478.004575
1255,4440.0,2997.383984,1442.616016
1874,32018.0,30598.312174,1419.687826
1470,0.0,1292.589322,1292.589322
1121,4368.0,3079.630094,1288.369906
395,6342.0,5105.730099,1236.269901


In [16]:
t=test[col]
ctb_pred=(ctb.predict(t))
sub = pd.DataFrame()
sub['ID'] = test['ID']
sub['power-generated']=ctb.predict(t)
sub['power-generated'].loc[sub['power-generated']<=0]  = 0
sub.to_csv("c.csv", index=False)
sub['power-generated'].describe()

count      877.000000
mean      6897.848482
std       9694.018378
min          0.000000
25%         16.022635
50%       1088.081496
75%      13961.256440
max      32982.159421
Name: power-generated, dtype: float64

In [15]:
pd.options.display.max_rows = 2200
#define a function to create variable importance dataframe
def get_catt_varimp(model, train_columns, max_vars=350):
    
    # Scikit-learn API LGBMClassifier or LGBMRegressor was fitted, 
    # so using feature_importances_ property
    feature_importances_perc = (model.feature_importances_ / sum(model.feature_importances_)) * 100
    cv_varimp_df = pd.DataFrame([train_columns, feature_importances_perc]).T

    cv_varimp_df.columns = ['feature_name', 'varimp (%)']

    cv_varimp_df.sort_values(by='varimp (%)', ascending=False, inplace=True)

    cv_varimp_df = cv_varimp_df.iloc[0:max_vars]   

    return cv_varimp_df
#
get_catt_varimp(ctb, X_train.columns)

,feature_name,varimp (%)
0,distance-to-solar-noon,62.911322
4,sky-cover,7.852569
6,humidity,7.605386
3,wind-speed,5.569763
7,average-wind-speed-(period),4.197106
8,average-pressure-(period),3.989863
2,wind-direction,3.975326
1,temperature,3.326049
5,visibility,0.572615
9,ind,0.0


In [225]:

import optuna
from catboost import CatBoostRegressor
X_train, X_valid, y_train, y_valid= train_test_split(X, y, test_size=0.2, random_state=123)
# Define the objective function for Optuna
def catboost_objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 10)
    }

    model = CatBoostRegressor(**params, silent=True)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=False)
    predictions = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, predictions, squared=False)

    return rmse

# Create an Optuna study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(catboost_objective, n_trials=100)

# Print the best hyperparameters and the corresponding RMSE
best_params = study.best_params
best_rmse = study.best_value
print(f"CatBoost Best Hyperparameters: {best_params}")
print(f"CatBoost Best RMSE: {best_rmse}")


[I 2024-02-02 16:57:15,190] A new study created in memory with name: no-name-ff2bba87-66af-48f8-9f81-c1059b1dfa53
[I 2024-02-02 16:57:38,410] Trial 0 finished with value: 3047.1740829516975 and parameters: {'learning_rate': 0.021276308072075215, 'n_estimators': 282, 'max_depth': 13, 'l2_leaf_reg': 0.004524945084399386, 'subsample': 0.6548365809627672, 'colsample_bylevel': 0.5841596074687128, 'min_child_samples': 2}. Best is trial 0 with value: 3047.1740829516975.
[I 2024-02-02 16:57:39,366] Trial 1 finished with value: 2982.264764164054 and parameters: {'learning_rate': 0.027057310669480295, 'n_estimators': 172, 'max_depth': 7, 'l2_leaf_reg': 3.728166415710105e-08, 'subsample': 0.705560899737624, 'colsample_bylevel': 0.6864362361154586, 'min_child_samples': 2}. Best is trial 1 with value: 2982.264764164054.
[I 2024-02-02 16:58:02,557] Trial 2 finished with value: 3205.4870729465183 and parameters: {'learning_rate': 0.2658360383737772, 'n_estimators': 168, 'max_depth': 15, 'l2_leaf_reg'

[I 2024-02-02 17:00:00,977] Trial 23 finished with value: 2960.381833656518 and parameters: {'learning_rate': 0.032278212893705346, 'n_estimators': 468, 'max_depth': 8, 'l2_leaf_reg': 1.5070977052493712e-07, 'subsample': 0.7517435780468866, 'colsample_bylevel': 0.9902794583209801, 'min_child_samples': 3}. Best is trial 11 with value: 2898.673497269353.
[I 2024-02-02 17:00:02,896] Trial 24 finished with value: 2982.672744337123 and parameters: {'learning_rate': 0.015724027199717088, 'n_estimators': 405, 'max_depth': 6, 'l2_leaf_reg': 1.5620919033657182e-07, 'subsample': 0.8521068956412047, 'colsample_bylevel': 0.9264046273675989, 'min_child_samples': 3}. Best is trial 11 with value: 2898.673497269353.
[I 2024-02-02 17:00:15,585] Trial 25 finished with value: 3028.7027388441397 and parameters: {'learning_rate': 0.01907945904372621, 'n_estimators': 465, 'max_depth': 11, 'l2_leaf_reg': 1.3115192366030814e-08, 'subsample': 0.9750722561396418, 'colsample_bylevel': 0.9909473444289882, 'min_ch

[I 2024-02-02 17:02:29,520] Trial 47 finished with value: 2962.0812484838557 and parameters: {'learning_rate': 0.027155158331361267, 'n_estimators': 373, 'max_depth': 10, 'l2_leaf_reg': 1.562069474885318e-06, 'subsample': 0.8020948645125392, 'colsample_bylevel': 0.9440219676251957, 'min_child_samples': 4}. Best is trial 11 with value: 2898.673497269353.
[I 2024-02-02 17:02:30,920] Trial 48 finished with value: 2993.2322817591257 and parameters: {'learning_rate': 0.04428742043283451, 'n_estimators': 283, 'max_depth': 8, 'l2_leaf_reg': 3.01120782417535e-07, 'subsample': 0.7486493722822225, 'colsample_bylevel': 0.8658063659593528, 'min_child_samples': 7}. Best is trial 11 with value: 2898.673497269353.
[I 2024-02-02 17:02:34,428] Trial 49 finished with value: 2935.1353936811365 and parameters: {'learning_rate': 0.022928076367527816, 'n_estimators': 352, 'max_depth': 9, 'l2_leaf_reg': 0.004341747025028416, 'subsample': 0.6925725897186966, 'colsample_bylevel': 0.9729355984982289, 'min_child

[I 2024-02-02 17:03:42,016] Trial 71 finished with value: 2984.0688096030026 and parameters: {'learning_rate': 0.03660022775888355, 'n_estimators': 470, 'max_depth': 11, 'l2_leaf_reg': 1.6420161829724252e-07, 'subsample': 0.7359870896968783, 'colsample_bylevel': 0.9640656915179769, 'min_child_samples': 3}. Best is trial 52 with value: 2884.9858575633502.
[I 2024-02-02 17:03:43,564] Trial 72 finished with value: 2937.3378271483266 and parameters: {'learning_rate': 0.05430231297311593, 'n_estimators': 482, 'max_depth': 9, 'l2_leaf_reg': 1.0241777066071803e-08, 'subsample': 0.7458695578112388, 'colsample_bylevel': 0.9976924087665934, 'min_child_samples': 4}. Best is trial 52 with value: 2884.9858575633502.
[I 2024-02-02 17:03:46,004] Trial 73 finished with value: 3053.2583051886904 and parameters: {'learning_rate': 0.04449581752585222, 'n_estimators': 456, 'max_depth': 10, 'l2_leaf_reg': 0.010828457907526682, 'subsample': 0.7658316040599183, 'colsample_bylevel': 0.9534190941426236, 'min_c

[I 2024-02-02 17:04:11,707] Trial 95 finished with value: 2962.8242962636555 and parameters: {'learning_rate': 0.02869035560436778, 'n_estimators': 329, 'max_depth': 6, 'l2_leaf_reg': 2.80513761107011e-06, 'subsample': 0.7425231043309207, 'colsample_bylevel': 0.601059174813677, 'min_child_samples': 6}. Best is trial 77 with value: 2842.2618090014284.
[I 2024-02-02 17:04:13,262] Trial 96 finished with value: 3005.525651559978 and parameters: {'learning_rate': 0.03261178318189405, 'n_estimators': 384, 'max_depth': 5, 'l2_leaf_reg': 9.312560608355543e-07, 'subsample': 0.7331179843546943, 'colsample_bylevel': 0.5460921877223209, 'min_child_samples': 5}. Best is trial 77 with value: 2842.2618090014284.
[I 2024-02-02 17:04:14,178] Trial 97 finished with value: 2937.347530680595 and parameters: {'learning_rate': 0.05379622932294692, 'n_estimators': 451, 'max_depth': 6, 'l2_leaf_reg': 5.229407627219036e-07, 'subsample': 0.7721922501624754, 'colsample_bylevel': 0.7345766371528604, 'min_child_sa

CatBoost Best Hyperparameters: {'learning_rate': 0.07835711563665773, 'n_estimators': 412, 'max_depth': 6, 'l2_leaf_reg': 1.067695159222943e-07, 'subsample': 0.8361107913175172, 'colsample_bylevel': 0.9682533530433258, 'min_child_samples': 5}
CatBoost Best RMSE: 2842.2618090014284


In [17]:
36
t_s={'learning_rate': 0.08210258622430408, 'n_estimators': 242, 'max_depth': 8, 'l2_leaf_reg': 0.01782898711705445, 'subsample': 0.9325000173819227, 'colsample_bylevel': 0.5697597984595163, 'min_child_samples': 6}

In [177]:
37
cl={'learning_rate': 0.05840299271922898, 'n_estimators': 407, 'max_depth': 9, 'l2_leaf_reg': 1.0323426492066644e-07, 'subsample': 0.8893594531228461, 'colsample_bylevel': 0.7719620546401684, 'min_child_samples': 10}

In [22]:
kf = KFold(n_splits=7,shuffle=False)
model = CatBoostRegressor(**t_s)
RMSE = []
R2=[]
c_train_pred = np.zeros(len(X))
c_test_pred = np.zeros(len(test))
for (train_index,test_index) in kf.split(X,y):
    X_train,X_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = y.iloc[train_index],y.iloc[test_index]

    model.fit(X_train,y_train,early_stopping_rounds=500,eval_set=[(X_test,y_test)],verbose=False)
    RMSE.append(np.sqrt(mean_squared_error(model.predict(X_test),y_test)))
    R2.append(r2_score(model.predict(X_test),y_test))

    c_train_pred+=model.predict(X)
    c_test_pred+=model.predict(test[col])
    c_predictions= np.absolute(c_test_pred/kf.n_splits)
    sub['power-generated']= np.clip(c_predictions, a_min=0, a_max=32018)
    sub.to_csv('ctb_cv7.csv',index=False)


print(
f"r2 or accuracy:     {np.mean(R2)}\n"       
f"Root Mean Squared Error: {np.mean(RMSE)} ")#2837

r2 or accuracy:     0.9083317686310498
Root Mean Squared Error: 2911.016674523887 


In [21]:
sub.describe()

,power-generated
count,877.000000
mean,81.714151
std,20.678198
min,0.234645
25%,90.000000
50%,90.000000
75%,90.000000
max,90.000000
